# Homework on MLE


In [1]:
using Distributions, Plots, LinearAlgebra, ForwardDiff, Optim
using DataFrames, CSV

#####   Estimate a Normal-Half Normal Model.

Consider the Model A below:

\begin{aligned}
 y_i & = \alpha + x_i' \beta + v_i - u_i,\\
 v_i & \sim N(0, \sigma_v^2), \\
 u_i & \sim N^+(0, \sigma_u^2).
\end{aligned}
 
 
The following is Model A's log-likelihood function of the $i$th observation. 

\begin{align}
L_i = - \ln \left(\frac{1}{2}\right) -\frac{1}{2}\ln (\sigma_v^2 + \sigma_u^2) + \ln
\phi\left(\frac{\epsilon_i}{\sqrt{\sigma_v^2 + \sigma_u^2}} \right) +
\ln \Phi\left(\frac{\mu_{*i}}{\sigma_*} \right),
\end{align}

where $\phi(z)$ and $\Phi(z)$ are the PDF and CDF of a standard normal distribution. Also,

\begin{aligned}
 \mu_{*i}  = \frac{-\sigma_u^2 \epsilon_i}{\sigma_v^2 + \sigma_u^2},\qquad
 \sigma_*^2  = \frac{\sigma_v^2  \sigma_u^2}{\sigma_v^2 + \sigma_u^2}. 
\end{aligned}


The attached dataset, `sampledata.csv`, contains data of agricultural production from India. The variables are the follows. They have been converted to appropriate units (taking log, etc.) so no further data processing is necessary.


|          |  |        |
|-------------------------------------|---|---------------------|
| yvar: rice output                   |   | Lland: land         |
| Plland: irrigated land              |   | Llabor: labor       |
| Lbull: bull cost                    |   | Lcost: other costs  |
| yr: production year                 |   | age: age of farmers |
| school: farmers' years of schooling |   | yr_1: same as year  |


The Indian farming model is the follows.

$yvar_i = \alpha + \beta_1 Lland_i + \beta_2 Plland_i + \beta_3 Llabor_i + \beta_4 Lbull_i + \beta_5 Lcost_i + \beta_6 yr_i + v_i - u_i$.
  
You may use the following code to read in the data.

```julia
####################
using DataFrames, CSV
df = DataFrame(CSV.File("sampledata.csv"))
y = df[:, "yvar"]       # the dep var
x = Matrix(df[:, 2:7])  # the indep vars, not including a constant
####################
```

###### Your work: 

- Based on (1), write a program of Model A's log-likelihood function and call it `NHN_mle`. You may use `loglike5` in the lecture note as the reference. 


- Use `NHN_mle` to estimate the Indian farming model. You may conduct the estimation using `Optim` or your own programs. The required result is a table with three columns: the 1st column is the coefficients, the 2nd column is the standard errors, and the 3rd column is the $t$ statistics. They do not have to be in DataFrames.


- The Estimation Guidelines:
  
  - You may use $-0.1$ as the initial values for all parameters. Or you may use the OLS result as the initial values for the $\alpha$ and $\beta$ coefficients (`x2=hcat(ones(size(y,1), 1), x); ols=inv(x2'x2)*(x2'y)`). Or, you may choose any initial values that seem to be reasonable choices. However, you _**cannot**_ use the true answer as the initial values.

  - I strongly suggest that your program uses the `autodiff = :forward` option (which uses automatic differentiation) in the estimation.
  
    - The `autodiff = :forward` option puts stringent requirements on data types which may not easy to work out. I suggest that you start with your program without the option (which would then default to numerical finite differences that is easier on the data). After you have a working program, you may add the option back and see if it works. Most likely you'll have error messages and you have to work out the issues. If you can't make it work, that's fine. Try your best.  
  
- Hint: Look at $|g(x)|$ to judge whether it is converged. It should be smaller than the convergence criterion. 

In [5]:
df = DataFrame(CSV.File("C:\\Users\\User\\sampledata.csv"))
y = df[:, "yvar"]     
x = Matrix(df[:, 2:7]) 
x2=hcat(ones(size(y,1), 1), x)
ols=inv(x2'x2)*(x2'y)

7-element Vector{Float64}:
  0.8773441654188616
  0.2735922714070149
  0.10895576192390166
  1.2625075239733157
 -0.5006703144428615
  0.0017781108365289011
  0.04915096010575404

In [6]:
noX=6
function NHN_mle(y, x, α, β, log_σᵥ², log_σᵤ²)
    σᵥ² = exp(log_σᵥ²)
    σᵤ² = exp(log_σᵤ²)
    ϵ  = y .- α .- x*β
    μ_star = (-σᵤ²*ϵ) / (σᵥ²+σᵤ²)

    llike = 0.0
    for i in eachindex(ϵ)
        σ_star = (σᵥ²*σᵤ²) / (σᵥ²+σᵤ²)
        llike += -log(0.5) - 0.5*log(σᵥ²+σᵤ²) + log(pdf(Normal(0,1), ϵ[i]/sqrt(σᵥ²+σᵤ²) )) + log(cdf(Normal(0,1), μ_star[i]/σ_star ))
    end
    return -llike
end
func = TwiceDifferentiable(vars -> NHN_mle(y, x, vars[1], vars[2:noX+1], vars[end-1], vars[end]),
                           ones(9); autodiff = :forward)

res = optimize(func, [ols; [-0.1,-0.1]], Newton(), Optim.Options(g_tol = 1e-9,iterations = 50) )    
_coevec = Optim.minimizer(res)
res_coeff = deepcopy(_coevec)                
res_coeff[end] = exp(_coevec[end])          
res_coeff[end-1] = exp(_coevec[end-1])  
 

res_Hessian  = Optim.hessian!(func, _coevec)  

var_cov_matrix = inv(res_Hessian)            
stderror  = sqrt.(diag(var_cov_matrix))
stderror[end] = res_coeff[end]*stderror[end]  
t_stats = res_coeff ./ stderror
res_table = hcat(res_coeff, stderror, t_stats)

println("The estimation table is")
res_table |> display

The estimation table is


9×3 Matrix{Float64}:
  1.59012    0.349898     4.54453
  0.286194   0.0698563    4.09689
  0.234652   0.17648      1.32963
  1.15546    0.0811715   14.2348
 -0.421433   0.0593928   -7.09569
  0.0066182  0.0130428    0.507422
  0.0340904  0.00803004   4.24536
  0.191584   0.162488     1.17907
  0.0974947  0.0593393    1.643

diff init values can not converge.

In [10]:
func = TwiceDifferentiable(vars -> NHN_mle(y, x, vars[1], vars[2], vars[3], vars[4], vars[5], vars[6], vars[7], vars[8], vars[9]),
                           ones(9); autodiff = :forward)

res2 = optimize(func, [-0.1 for i in 1:9], Newton(), Optim.Options(g_tol = 1e-9,iterations = 50) )

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     1.083142e+02

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 2.26e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.61e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.26e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.08e-08 ≰ 0.0e+00
    |g(x)|                 = 2.46e+02 ≰ 1.0e-09

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    209
    ∇f(x) calls:   209
    ∇²f(x) calls:  51
